In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path
import neptune # for logging progress and results
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures
neptune_kwargs = {'project':"commander.pho/PhoDibaLongShort2023",
'api_token':"eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxOGIxODU2My1lZTNhLTQ2ZWMtOTkzNS02ZTRmNzM5YmNjNjIifQ=="}

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
from attrs import define, field, fields, Factory

# Pho's Formatting Preferences
# from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
# set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.general_helpers import CodeConversion
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import print_keys_if_possible, print_value_overview_only, document_active_variables, objsize, print_object_memory_usage, debug_dump_object_member_shapes, TypePrintMode
from pyphocorehelpers.print_helpers import get_now_day_str, get_now_time_str, get_now_time_precise_str
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType

from pyphocorehelpers.print_helpers import generate_html_string


# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot, create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, SessionBatchProgress, batch_programmatic_figures, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
from pyphoplacecellanalysis.Pho2D.PyQtPlots.Extensions.pyqtgraph_helpers import build_pyqtgraph_epoch_indicator_regions # Add session indicators to pyqtgraph plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _helper_add_long_short_session_indicator_regions

import pyphoplacecellanalysis.External.pyqtgraph as pg
# 'time_bin_indices': valid_time_bin_indicies, 'posterior_to_pf_mean_surprise': one_left_out_posterior_to_pf_surprises_mean, 'posterior_to_scrambled_pf_mean_surprise': one_left_out_posterior_to_scrambled_pf_surprises_mean}
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_expected_vs_observed_firing_rates
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_long_short_surprise_difference_plot

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import _update_pipeline_missing_preprocessing_parameters
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import perform_write_to_file

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots

from scripts.run_BatchAnalysis import post_compute_validate, _on_complete_success_execution_session

session_batch_status = {}
session_batch_errors = {}
enable_saving_to_disk = False

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'

## Data must be pre-processed using the MATLAB script located here: 
#     neuropy/data_session_pre_processing_scripts/KDIBA/IIDataMat_Export_ToPython_2022_08_01.m
# From pre-computed .mat files:

local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=self.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

good_contexts_list = [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-17_12-33-47'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_13-6-1'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-18_15-23-32'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-27_14-43-12'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_12-48-38'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-11_16-2-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_14-49-24'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-19_13-50-7'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-26_13-51-50'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_21-26-8'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-05_19-26-43'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_21-17-16'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-04_21-20-3')]


In [3]:
%pdb off
# %%viztracer
# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54')
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25')

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
# saving_mode = PipelineSavingScheme.SKIP_SAVING
# force_reload = False

# Force write:
saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
force_reload = True

    # ==================================================================================================================== #
    # Load Pipeline                                                                                                        #
    # ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation', 
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=True) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'

if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        print(f'cannot load global results: {e}')

# 6m 1.1s
# 12m 15.6s


Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25
Skipping loading from pickled file because force_reload == True.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.epochs_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.position_info.mat... done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.spikes.mat... 

C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\session\Formats\SessionSpecifications.py:140: UserWarning: WARNING: Optional File: W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


done.
Failure loading .position.npy. Must recompute.

Saving updated position results results : W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.position.npy... 2006-6-08_21-16-25.position.npy saved
done.
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.
	 Saving updated interpolated spike position results results : W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.interpolated_spike_positions.npy... 2006-6-08_21-16-25.interpolated_spike_positions.npy saved
done.
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.laps_info.mat... done.
setting laps object.
session.laps loaded successfully!
Loading matlab import file results : W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\2006-6-08_21-16-25.replay_info.mat... done.
session.replays loaded successfully!
Loading success: W:\Data\KDIBA\gor01\two\2006-6-08_21-16-

C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to b

Applying session filter named "maze2"...
Constraining to epoch with times (start: 722.653951405664, end: 1201.0839364149142)
computing neurons mua for session...

Applying session filter named "maze"...
Constraining to epoch with times (start: 0.0, end: 1201.0839364149142)
computing neurons mua for session...



C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['lap_id']] = laps_df[['lap_id']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  laps_df[['start_spike_index', 'end_spike_index']] = laps_df[['start_spike_index', 'end_spike_index']].astype('int')
C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy\core\laps.py:70: SettingWithCopyWarning: 
A value is trying to b

using provided computation_functions_name_whitelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
due to whitelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (15.182688599114876, 256.24088442352786)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493))
	 done.
	 spikes_df[time_variable_name]: (19348,) should be less than time_window_edges: (20071,)!
	 spikes_df[time_variable_name]: (19348,) should be less than time_window_edges: (20071,)!


C:\Users\pho\repos\Spike3DWorkEnv\pyPhoPlaceCellAnalysis\src\pyphoplacecellanalysis\Analysis\Decoder\reconstruction.py:363: RuntimeWarning: divide by zero encountered in divide
  return C * np.exp(numerator/denominator)


two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (20269,)
updating computation_results...
done.
due to whitelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (15.182688599114876, 256.24088442352786)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493))
	 done.
two_step_decoder_result['most_likely_position_flat_max_likelihood_values'].shape = (12894,)
updating computation_results...
done.
due to whitelist, including only 6 out of 16 registered computation functions.
Recomputing active_epoch_placefields... using self.config.grid_bin_bounds_1D: (15.182688599114876, 256.24088442352786)
	 done.
Recomputing active_epoch_placefields2D... using self.config.grid_bin_bounds: ((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493)

In [4]:
## Post Compute Validate 2023-05-16:
from scripts.run_BatchAnalysis import post_compute_validate

post_compute_validate(curr_active_pipeline)

were pipeline preprocessing parameters missing and updated?: False


In [5]:
curr_active_pipeline.reload_default_computation_functions()
extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'] # do only specifiedl , 'long_short_rate_remapping'
force_recompute_global = force_reload
# extended_computations_include_includelist=['long_short_post_decoding'] # do only specifiedl
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
if len(newly_computed_values) > 0:
	print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
	try:
		# curr_active_pipeline.global_computation_results.persist_time = datetime.now()
		# Try to write out the global computation function results:
		curr_active_pipeline.save_global_computation_results()
	except Exception as e:
		print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
		print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
else:
	print(f'no changes in global results.')
# 10m 29.5s

included whitelist is specified: ['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding'], so only performing these extended computations.
Running batch_extended_computations(...) with global_epoch_name: "maze"
jonathan_firing_rate_analysis missing.
	 Recomputing jonathan_firing_rate_analysis...
	 done.
long_short_fr_indicies_analyses missing.
	 Recomputing long_short_fr_indicies_analyses...
	 done.
long_short_decoding_analyses missing.
	 Recomputing long_short_decoding_analyses...
sane_midpoint_x: 135.71178651132138, hardcoded_track_midpoint_x: None, track_min_max_x: (15.182688599114876, 256.24088442352786)
hardcoded_track_midpoint_x is None, falling back to sane_midpoint_x... 135.71178651132138
desc_crossings_x: (20,), asc_crossings_x: (20,)
setting new computation epochs because laps changed.
using self.config.grid_bin_bounds_1D: (15.182688599114876, 256.24088442352786)
using self.config.grid_bin_bounds: ((15.18

In [6]:
curr_active_pipeline.reload_default_display_functions()
neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)


https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/
https://app.neptune.ai/commander.pho/PhoDibaLongShortUpdated/e/LS2023-73
shared_aclus: [2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 102]
long_only_aclus: [19, 23, 36, 54, 60, 78, 101, 103]
short_only_aclus: [8, 82]
curr_session_parent_out_path: C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\Screenshots\ProgrammaticDisplayFunctionTesting\2023-06-13\kdiba\gor01\two\2006-6-08_21-16-25
shared_aclus: [2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 61, 62, 63, 64,

# Get output files:

In [ ]:
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatBaseRegisteredClass
from neuropy.core.session.KnownDataSessionTypeProperties import KnownDataSessionTypeProperties
from neuropy.core.session.dataSession import DataSession
from neuropy.core.session.Formats.SessionSpecifications import SessionFolderSpec, SessionFileSpec

# 'ripple_df.pkl', .ripple.npy'

# files_list = [curr_active_pipeline.sess.filePrefix.with_suffix(a_suffix) for a_suffix in session_folder_output_file_suffix_list]
# files_list

"""
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.interpolated_spike_positions.npy,"2023-06-07 01:03:36",35290406
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.mua.npy,"2023-06-07 01:03:37",7766554
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.pbe.npy,"2023-06-07 01:03:37",10902
KDIBA/pin01/one/11-02_17-46-44/11-02_17-46-44.position.npy,"2023-06-07 01:03:36",2446247
"""

session_name = curr_active_pipeline.session_name
print(f'session_name: {session_name}')
output_folder_spec: SessionFolderSpec = SessionFolderSpec(required=[
        SessionFileSpec('loadedSessPickle.pkl', session_name, 'The primary .pkl file after computing the session', None),
        SessionFileSpec('{}.interpolated_spike_positions.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.mua.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.pbe.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('{}.position.npy', session_name, 'A cached initial session computation', None),
        SessionFileSpec('output/global_computation_results.pkl', session_name, 'The global_computation_results computed for the pipeline.', None),
    ],
    optional=[
        SessionFileSpec('output/leave_one_out_results_short.pkl', session_name, 'Optional caching file.', None),
        SessionFileSpec('output/leave_one_out_surprise_results_long.pkl', session_name, 'Optional caching file', None),
        SessionFileSpec('ripple_df.pkl', session_name, 'SWR ripples computed by an external script from another repo.', None),
        SessionFileSpec('{}.ripple.npy', session_name, 'Ripples computed by this repo and cached to the directory.', None),
    ]
)


is_resolved, resolved_required_filespecs_dict, resolved_optional_filespecs_dict = output_folder_spec.validate(curr_active_pipeline.sess.basepath)

if is_resolved:
    print(f'resolved_required_filespecs_dict: {resolved_required_filespecs_dict}')
    

# out_dict = {}
# out_dict['required_files'] = [str(a_path) for a_path in resolved_required_filespecs_dict.keys() if a_path.resolve().exists()]
# out_dict['optional_files'] = [str(a_path) for a_path in resolved_optional_filespecs_dict.keys() if a_path.resolve().exists()]
# out_dict

# Get the list of strings for all output files that actually exist:
all_output_file_paths: list[str] = [str(a_path) for a_path in resolved_required_filespecs_dict.keys() if a_path.resolve().exists()] + [str(a_path) for a_path in resolved_optional_filespecs_dict.keys() if a_path.resolve().exists()]
all_output_file_paths


In [ ]:

# curr_active_pipeline.register_output_file(output_path=output_path, 
curr_active_pipeline.registered_output_files_list
curr_active_pipeline.registered_output_files
# curr_active_pipeline.get_output_path()

# reveal_in_system_file_manager(curr_active_pipeline.get_daily_programmatic_session_output_path())

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import loadData, saveData

# Save in case something goes wrong:
temp_out_path = 'registered_output_files.pkl'


In [ ]:
saveData(temp_out_path, (curr_active_pipeline.registered_output_files.to_dict()))
reveal_in_system_file_manager(temp_out_path)

In [ ]:
test_registered_output_files = loadData(temp_out_path)
test_registered_output_files
curr_active_pipeline.registered_output_files = test_registered_output_files

In [ ]:
# Neptune class:
import neptune
from neptune.types import File
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner
neptuner = Neptuner.init_with_pipeline(curr_active_pipeline)
neptuner

In [ ]:
# succeeded_fig_paths
succeeded_fig_paths, failed_fig_paths = neptuner.upload_figures(curr_active_pipeline)

In [ ]:
# Set up all of the session properties as run context.
# run['session'] = curr_active_pipeline.get_session_context()
neptuner.stop()

### Pipeline Loading/Saving

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.TEMP_THEN_OVERWRITE) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
# TypeError: cannot pickle 'MplMultiTab' object
# PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell
# TypeError: cannot pickle 'PyQt5.QtCore.pyqtSignal' object

In [ ]:
curr_active_pipeline.save_pipeline(saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE)

In [ ]:
curr_active_pipeline.save_global_computation_results() # PicklingError: Can't pickle .set_closure_cell at 0x000002BF248F50D0>: it's not found as attr._compat.make_set_closure_cell..set_closure_cell

In [ ]:
curr_active_pipeline.load_pickled_global_computation_results()

In [ ]:
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
curr_active_pipeline.save_global_computation_results()

# End Run

In [7]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333

## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']


In [8]:
long_grid_bin_bounds, short_grid_bin_bounds, global_grid_bin_bounds = [curr_active_pipeline.computation_results[a_name].computation_config['pf_params'].grid_bin_bounds for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
print(long_grid_bin_bounds, short_grid_bin_bounds, global_grid_bin_bounds)
long_pos_df, short_pos_df, global_pos_df, all_pos_df = [a_sess.position.to_dataframe() for a_sess in (long_session, short_session, global_session, curr_active_pipeline.sess)]
combined_pos_df = deepcopy(all_pos_df)
combined_pos_df.loc[long_pos_df.index, 'long_lin_pos'] = long_pos_df.lin_pos
combined_pos_df.loc[short_pos_df.index, 'short_pos_df'] = short_pos_df.lin_pos
combined_pos_df.loc[global_pos_df.index, 'global_lin_pos'] = global_pos_df.lin_pos

# ['long_lin_pos', 'short_pos_df', 'global_lin_pos']
combined_pos_df
%matplotlib qt
combined_pos_df.plot(x='t', y=['x', 'lin_pos', 'long_lin_pos', 'short_pos_df', 'global_lin_pos'])

((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493)) ((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493)) ((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493))


<Axes: xlabel='t'>

In [ ]:
# Clearing 'long_short_post_decoding'
# del curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
# drop_columns = [col for col in df2.columns if col in df1.columns]
# df1 = df1.drop(drop_columns, axis=1)

In [9]:
long_pf1D.plot_ratemaps_1D()

(<Axes: title={'center': '1D Placemaps Placemaps (60 good cells)'}, xlabel='Position'>,
 array([52, 51, 58, 27, 33, 12, 31, 14, 39, 21, 20, 38,  4,  0, 23, 35, 46,
        30, 49, 57,  5,  2, 42, 59, 34, 44, 22, 11, 15, 32, 28, 19, 48,  3,
        18,  9, 26,  6, 36, 10, 29, 43, 41, 17, 54, 53, 55, 56, 45,  7, 40,
        24, 25, 37,  1, 47, 16, 13, 50,  8], dtype=int64),
 array([[0.51764706, 0.57794118, 0.63823529, 0.69852941, 0.75882353,
         0.81911765, 0.87941176, 0.93970588, 1.        , 0.69803922,
         0.73578431, 0.77352941, 0.81127451, 0.84901961, 0.88676471,
         0.9245098 , 0.9622549 , 1.        , 0.54901961, 0.60539216,
         0.71813725, 0.7745098 , 0.83088235, 0.8872549 , 0.94362745,
         1.        , 0.38823529, 0.44845291, 0.51024503, 0.57361166,
         0.63855281, 0.70506846, 0.77315863, 0.84282331, 0.9140625 ,
         0.22352941, 0.2725518 , 0.32637032, 0.38498496, 0.44839572,
         0.58960561, 0.66740475, 0.75      , 0.22352941, 0.27330685,
    

In [ ]:
long_pf2D.plot_ratemaps_2D()

In [ ]:
# long_pf2D
# global_pf2D.bin_info
global_pf1D.bin_info

# 2023-05-26 - Expected vs Observed firing rates
## 2023-05-30 Comments from Kourosh:
- concern: is it valid to use the most-likely predicted position for each decoder for this? Isn't it a bit circular because the firing rates determine the likelihood of the position?
- sign-rank is equivalent of paired t-test for non-parametric, use that to compare the means for each cell after averaging over replays to remove the noise from that
- OPTION: could add a continuous plot that shows the difference between the two decoders and the animal's measured position as a function of time (observed_x - long_decoded_x), (observed_x - short_decoded_x).
- [x] Plot just replay-to-replay series, removing the time axis

In [ ]:
## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']

num_epochs: int = len(num_timebins_in_epoch)

## 2023-05-30 - `plot_expected_vs_observed`

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _prepare_plot_expected_vs_observed

fig, axes, final_context, active_out_figure_paths = _prepare_plot_expected_vs_observed(curr_active_pipeline, defer_render=False)

In [ ]:
## 2023-05-31 - Reference Output of matplotlib figure to file, along with building appropriate context.
active_session_figures_out_path = curr_active_pipeline.get_daily_programmatic_session_output_path()
final_context = curr_active_pipeline.sess.get_context().adding_context('display_fn', display_fn_name='plot_expected_vs_observed').adding_context('display_kwargs', **display_kwargs)
print(f'final_context: {final_context}')
active_out_figure_paths = perform_write_to_file(fig, final_context, figures_parent_out_path=active_session_figures_out_path, register_output_file_fn=curr_active_pipeline.register_output_file)


In [ ]:
all_epochs_computed_observed_from_expected_difference_LONG[0] #.shape (42, 8)

In [ ]:
all_epochs_computed_observed_from_expected_difference_maximum_LONG[0] # .shape (42, 8)

In [ ]:
all_epochs_computed_expected_cell_firing_rates_mean_LONG[0] # .shape (42,)

In [ ]:
all_epochs_computed_observed_from_expected_difference_LONG[0].shape

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())

In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf

In [ ]:
jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins #.shape

In [ ]:
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
instantaneous_fr_df = deepcopy(jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values)
instantaneous_fr_df['t'] = jonathan_firing_rate_analysis_result.time_binned_instantaneous_unit_specific_spike_rate.time_bins
instantaneous_fr_df #.plot()

smoothed_instantaneous_fr_df = instantaneous_fr_df.ewm(span=60).mean()

smoothed_instantaneous_fr_df.plot(x='t')
plt.title("Instantaneous Firing Rates")
plt.xlabel('time')
plt.ylabel('Firing Rate (Hz)')

In [ ]:
import matplotlib
%matplotlib qt
import matplotlib.pyplot as plt
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_figure_basename_from_display_context
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_session_long_short_track_firing_rate_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
long_plots, short_plots = _plot_session_long_short_track_firing_rate_figures(curr_active_pipeline, jonathan_firing_rate_analysis_result, figures_parent_out_path=curr_active_pipeline.get_output_path())

In [ ]:
instantaneous_fr_df.set_index('t').to_clipboard()

In [ ]:
## Require placefield presence on either the long or the short
jonathan_firing_rate_analysis_result.neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])]

In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()

# 2023-05-19 - Testing S-only emergence, L-only replays in S, peak position remappings 

In [10]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

jonathan_firing_rate_analysis_result = JonathanFiringRateAnalysisResult(**curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.to_dict())
neuron_replay_stats_df = jonathan_firing_rate_analysis_result.neuron_replay_stats_df.copy()
neuron_replay_stats_df

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
2,48.182689,True,74.182689,True,False,SplitPartitionMembership.SHARED,1.692746,1.254248,-0.438498,12.958907,15.386818,2.427911,7.325826,8.320533,0.994707,0,34,11,23,NeuronType.PYRAMIDAL
3,224.182689,True,172.182689,True,False,SplitPartitionMembership.SHARED,0.734391,1.449838,0.715447,9.148518,10.480347,1.331828,4.941455,5.965092,1.023638,1,26,11,15,NeuronType.PYRAMIDAL
4,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,0.131348,1.440034,1.308686,7.948878,8.812144,0.863267,4.040113,5.126089,1.085976,2,9,4,5,NeuronType.INTERNEURONS
5,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,14.315492,19.126139,4.810646,45.657137,48.967996,3.310859,29.986315,34.047067,4.060753,3,77,30,47,NeuronType.CONTAMINATED
6,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,2.567909,1.696307,-0.871602,16.090167,7.747413,-8.342754,9.329038,4.721860,-4.607178,4,16,7,9,NeuronType.INTERNEURONS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,16.403172,14.866414,-1.536758,47.332661,47.784624,0.451962,31.867917,31.325519,-0.542398,97,77,30,47,NeuronType.CONTAMINATED
100,60.182689,True,74.182689,True,False,SplitPartitionMembership.SHARED,2.116158,3.600593,1.484435,12.131697,10.231220,-1.900477,7.123927,6.915906,-0.208021,98,33,17,16,NeuronType.PYRAMIDAL
101,24.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,2.718215,0.919096,-1.799119,18.019961,19.150225,1.130264,10.369088,10.034660,-0.334427,99,33,19,14,NeuronType.PYRAMIDAL
102,NaN,False,NaN,False,True,SplitPartitionMembership.SHARED,3.724971,3.439583,-0.285389,12.541263,11.500480,-1.040783,8.133117,7.470031,-0.663086,100,51,20,31,NeuronType.CONTAMINATED


In [11]:

from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership, SetPartition
## 2023-05-19 - Get S-only pfs
is_S_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_long_pf']), neuron_replay_stats_df['has_short_pf'])
is_S_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.RIGHT_ONLY
assert (is_S_pf_only == is_S_only).all()
S_only_aclus = neuron_replay_stats_df.index[is_S_only].to_numpy()
neuron_replay_stats_df[is_S_pf_only]


,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
8,NaN,False,194.182689,True,True,SplitPartitionMembership.RIGHT_ONLY,0.466354,2.154300,1.687945,11.839761,23.905534,12.065773,6.153058,13.029917,6.876859,6,8,3,5,NeuronType.PYRAMIDAL
82,NaN,False,76.182689,True,True,SplitPartitionMembership.RIGHT_ONLY,0.561427,1.126722,0.565295,6.406403,7.820815,1.414412,3.483915,4.473769,0.989854,80,8,5,3,NeuronType.PYRAMIDAL


In [12]:

## Show L-only pfs stop replaying on S
is_L_pf_only = np.logical_and(np.logical_not(neuron_replay_stats_df['has_short_pf']), neuron_replay_stats_df['has_long_pf'])
is_L_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.LEFT_ONLY
assert (is_L_pf_only == is_L_only).all()
L_only_aclus = neuron_replay_stats_df.index[is_L_only].to_numpy()
neuron_replay_stats_df[is_L_only]


## For ('kdiba', 'gor01', 'one', '2006-6-09_1-22-43') - Have L-only cells [24, 98] that have ['short_num_replays'] = [8, 7]. We were hoping that there would be few to no replays on the S-track that involved L-only cells.

,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,short_replay_mean,replay_diff,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type
19,244.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.578695,0.359582,-0.219113,35.395542,37.108055,1.712513,17.987118,18.733819,0.746700,17,5,1,4,NeuronType.PYRAMIDAL
23,196.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.337363,0.239914,-0.097450,7.899525,4.128881,-3.770643,4.118444,2.184397,-1.934046,21,4,3,1,NeuronType.PYRAMIDAL
36,240.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,2.357836,0.976005,-1.381831,31.461754,28.210910,-3.250843,16.909795,14.593458,-2.316337,34,8,2,6,NeuronType.PYRAMIDAL
54,36.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.417598,0.238529,-0.179069,11.865386,7.415215,-4.450171,6.141492,3.826872,-2.314620,52,4,2,2,NeuronType.PYRAMIDAL
60,38.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.226288,0.221050,-0.005238,6.877918,6.518489,-0.359429,3.552103,3.369770,-0.182333,58,7,5,2,NeuronType.PYRAMIDAL
78,56.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.264437,0.312520,0.048083,6.226899,9.641232,3.414334,3.245668,4.976876,1.731208,76,20,11,9,NeuronType.PYRAMIDAL
101,24.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,2.718215,0.919096,-1.799119,18.019961,19.150225,1.130264,10.369088,10.034660,-0.334427,99,33,19,14,NeuronType.PYRAMIDAL
103,66.182689,True,NaN,False,True,SplitPartitionMembership.LEFT_ONLY,0.782801,0.857430,0.074630,11.693584,13.215135,1.521550,6.238193,7.036282,0.798090,101,20,13,7,NeuronType.PYRAMIDAL


In [13]:
## 2023-05-23 - Get Common (SHARED) placefields
## Goal 1: From the cells with the placefields on both tracks, compute the degree to which they remap in position and sort them according to their distance.
is_BOTH_pf_only = np.logical_and(neuron_replay_stats_df['has_short_pf'], neuron_replay_stats_df['has_long_pf']) # (63,)
BOTH_pf_only_aclus = neuron_replay_stats_df.index[is_BOTH_pf_only].to_numpy()

## NOTE: is_BOTH_pf_only is a much more stringent requirement (and a strict subset) than `is_BOTH_only`
is_BOTH_only = neuron_replay_stats_df.track_membership == SplitPartitionMembership.SHARED # (99,)
BOTH_only_aclus = neuron_replay_stats_df.index[is_BOTH_only].to_numpy()
assert BOTH_only_aclus.shape[0] >= BOTH_pf_only_aclus.shape[0]

BOTH_pf_only_df = neuron_replay_stats_df[is_BOTH_pf_only].copy()
BOTH_pf_only_df['long_short_pf_peak_x_displacement'] = BOTH_pf_only_df['long_pf_peak_x'].values - BOTH_pf_only_df['short_pf_peak_x'].values
BOTH_pf_only_df['long_short_pf_peak_x_distance'] = BOTH_pf_only_df['long_short_pf_peak_x_displacement'].abs()
BOTH_pf_only_df.sort_values(by=['long_short_pf_peak_x_distance'], inplace=True, ascending=False)
BOTH_pf_only_df

#TODO 2023-05-23 - Can do more detailed peaks analysis with: long_results.RatemapPeaksAnalysis and short_results.RatemapPeaksAnalysis



,long_pf_peak_x,has_long_pf,short_pf_peak_x,has_short_pf,has_na,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,long_replay_mean,...,long_mean,short_mean,mean_diff,neuron_IDX,num_replays,long_num_replays,short_num_replays,neuron_type,long_short_pf_peak_x_displacement,long_short_pf_peak_x_distance
89,242.182689,True,48.182689,True,False,SplitPartitionMembership.SHARED,2.501139,1.699489,-0.801651,16.938301,...,9.719720,7.401019,-2.318701,87,38,16,22,NeuronType.PYRAMIDAL,194.0,194.0
29,240.182689,True,54.182689,True,False,SplitPartitionMembership.SHARED,2.072015,1.520343,-0.551672,22.543239,...,12.307627,12.456593,0.148966,27,9,3,6,NeuronType.PYRAMIDAL,186.0,186.0
83,226.182689,True,62.182689,True,False,SplitPartitionMembership.SHARED,3.435349,3.079212,-0.356137,24.612638,...,14.023994,9.503924,-4.520069,81,14,4,10,NeuronType.PYRAMIDAL,164.0,164.0
94,208.182689,True,52.182689,True,False,SplitPartitionMembership.SHARED,1.912620,1.125598,-0.787023,12.330258,...,7.121439,7.259112,0.137673,92,22,11,11,NeuronType.PYRAMIDAL,156.0,156.0
22,184.182689,True,48.182689,True,False,SplitPartitionMembership.SHARED,3.120983,3.017038,-0.103945,8.185430,...,5.653206,6.828374,1.175168,20,29,11,18,NeuronType.PYRAMIDAL,136.0,136.0
59,56.182689,True,188.182689,True,False,SplitPartitionMembership.SHARED,1.031382,1.100756,0.069374,NaN,...,NaN,9.046174,NaN,57,2,0,2,NeuronType.PYRAMIDAL,-132.0,132.0
9,182.182689,True,50.182689,True,False,SplitPartitionMembership.SHARED,3.375154,1.778960,-1.596194,9.875185,...,6.625170,6.194713,-0.430456,7,12,6,6,NeuronType.PYRAMIDAL,132.0,132.0
39,176.182689,True,76.182689,True,False,SplitPartitionMembership.SHARED,0.319523,0.309076,-0.010447,8.089984,...,4.204753,3.575643,-0.629111,37,12,3,9,NeuronType.PYRAMIDAL,100.0,100.0
24,170.182689,True,70.182689,True,False,SplitPartitionMembership.SHARED,0.692162,0.908367,0.216205,11.522505,...,6.107334,5.947835,-0.159498,22,27,13,14,NeuronType.PYRAMIDAL,100.0,100.0
61,172.182689,True,74.182689,True,False,SplitPartitionMembership.SHARED,0.430736,1.531943,1.101206,8.596873,...,4.513805,8.085797,3.571992,59,24,10,14,NeuronType.PYRAMIDAL,98.0,98.0


In [ ]:
""" 24 cells, 133 Epochs, 789 Total Timebins
# Other
	
	- is_non_firing_time_bin: numpy.ndarray - (24, 789)
	- all_epochs_num_epoch_time_bins: numpy.ndarray - (133,)
	
    ## Indexing Helpers:
		- all_epochs_reverse_flat_epoch_indicies_array: numpy.ndarray - (789,)
		- split_by_epoch_reverse_flattened_time_bin_indicies: list - (133,)
    
	- original_1D_decoder: pyphoplacecellanalysis.Analysis.Decoder.reconstruction.BasePositionDecoder
		- pf: neuropy.analyses.placefields.PfND
		- neuron_IDXs: numpy.ndarray - (24,)
		- neuron_IDs: numpy.ndarray - (24,)
		- F: numpy.ndarray - (119, 24)
		- P_x: numpy.ndarray - (119, 1)
        
	- flat_all_epochs_decoded_epoch_time_bins: numpy.ndarray - (24, 789)

# Measured
	- flat_all_epochs_measured_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - (24, 789)


# Expected
	- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_spike_counts: numpy.ndarray - (24, 789)
	- flat_all_epochs_difference_from_expected_cell_firing_rates: numpy.ndarray - (24, 789)
    
    
    
## Epoch-based
	- all_epochs_decoded_epoch_time_bins_mean: numpy.ndarray - (133, 24)

    
    
    
- flat_all_epochs_measured_cell_firing_rates: numpy.ndarray - .shape: (24, 789) 
- flat_all_epochs_computed_expected_cell_firing_rates: numpy.ndarray - .shape: (24, 789)


"""

In [ ]:
long_results_obj.flat_all_epochs_computed_expected_cell_firing_rates
long_results_obj.flat_all_epochs_measured_cell_firing_rates

# long_results_obj.result_df_grouped

measured_dfs = pd.DataFrame(long_results_obj.flat_all_epochs_measured_cell_firing_rates.T, columns=long_results_obj.original_1D_decoder.neuron_IDs)
expected_dfs = pd.DataFrame(long_results_obj.flat_all_epochs_computed_expected_cell_firing_rates.T, columns=long_results_obj.original_1D_decoder.neuron_IDs)
# pd.DataFrame(np.stack((long_results_obj.flat_all_epochs_measured_cell_firing_rates, long_results_obj.flat_all_epochs_computed_expected_cell_firing_rates)), columns=['measured_fr', 'expected_fr'])
measured_dfs.plot()


In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df.to_clipboard()

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import compute_evening_morning_parition

difference_sorted_aclus, evening_sorted_aclus, morning_sorted_aclus = compute_evening_morning_parition(neuron_replay_stats_df, debug_print=True)
sorted_neuron_replay_stats_df = neuron_replay_stats_df.reindex(difference_sorted_aclus).copy() # This seems to work to re-sort the dataframe by the sort indicies
sorted_neuron_replay_stats_df

In [ ]:
from neuropy.core.neurons import NeuronType

%matplotlib qt
# Look at replays during ripples vs. those not during ripples. Also potentially PBEs vs. not PBEs.

# NeuronType.from_any_string_series(['pyr','intr'])
'pyr','cont','intr'

active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['pyr'])) # only the pyramidal neurons

In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:
curr_active_pipeline.display('_display_jonathan_interactive_replay_firing_rate_comparison', active_identifying_session_ctx, included_neuron_types=NeuronType.from_any_string_series(['intr']), require_placefield=False) # only the pyramidal neurons

In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import build_replays_custom_scatter_markers, CustomScatterMarkerMode

_curr_included_aclus = jonathan_firing_rate_analysis_result.neuron_replay_stats_df[np.logical_or(jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_long_pf'], jonathan_firing_rate_analysis_result.neuron_replay_stats_df['has_short_pf'])].index.to_numpy()
_curr_output = curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison', active_identifying_session_ctx, included_unit_neuron_IDs=_curr_included_aclus, marker_split_mode=CustomScatterMarkerMode.NoSplit)

## 2023-04-13 - Shuffled Surprise
""" 
Relevant Functions:
`perform_full_session_leave_one_out_decoding_analysis`:
	`perform_leave_one_aclu_out_decoding_analysis`:	from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import perform_leave_one_aclu_out_decoding_analysis
	`_analyze_leave_one_out_decoding_results`: from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.DefaultComputationFunctions import _analyze_leave_one_out_decoding_results
"""


In [ ]:
# 2023-04-18 - Refactored into decoder_result
result, result_df, result_df_grouped = long_results_obj.result, long_results_obj.result_df, long_results_obj.result_df_grouped

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:

# 1. Get set of cells active in a given time bin, for each compute the surprise of its placefield with the leave-one-out decoded posterior.

# 2. From the remainder of cells (those not active), randomly choose one to grab the placefield of and compute the surprise with that and the same posterior.

# Expectation: The cells that are included in the time bin are expected to have a lower surprise (be less correlated with) the posterior.

In [ ]:
win, plots = plot_long_short_surprise_difference_plot(curr_active_pipeline, long_results_obj, short_results_obj, long_epoch_name, short_epoch_name)


In [ ]:
long_results_obj.timebinned_neuron_info.n_timebins # 736
short_results_obj.timebinned_neuron_info.n_timebins

In [ ]:
long_results_obj.flat_all_epochs_measured_cell_firing_rates.shape # (30, 736)
# short_results_obj.flat_all_epochs_measured_cell_firing_rates.shape # (30, 736)



In [ ]:
win, plots_tuple, legend = plot_long_short_expected_vs_observed_firing_rates(long_results_obj, short_results_obj, limit_aclus=list(BOTH_pf_only_df.index[:2].values))
long_epoch_indicator_region_items, short_epoch_indicator_region_items = _helper_add_long_short_session_indicator_regions(win, long_epoch=curr_active_pipeline.filtered_epochs[long_epoch_name], short_epoch=curr_active_pipeline.filtered_epochs[short_epoch_name])

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot # works pretty well seemingly

export_pyqtgraph_plot(win)
# pg.setConfigOption('leftTitle', 'MathText')
# win.setTitle(r'JSD(p_x_given_n, pf[<font size="4"><b><span style="color:red;">i</span></b></font>]) - JSD(p_x_given_n, pf[<font size="4"><b>j</b></font>]) where <font size="4"><b>j</b></font> non-firing')

# win.setTitle(generate_html_string(f'JSD(p_x_given_n, pf[{i_str}]) - JSD(p_x_given_n, pf[{j_str}]) where {j_str} non-firing', font_size=8))

# r'$\fn{JSD}{\matr{pf}_{i}, \Pr{\vec{x}_{t}|\vec{n}_{t}}} - \fn{JSD}{\matr{pf}_{j}, \Pr{\vec{x}_{t}|\vec{n}_{t}}}$'
# title_str

## Pre 2023-04-13

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib qt

from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure

from neuropy.core.neurons import NeuronType
# # Include only pyramidal aclus:
# print(f'all shared_aclus: {len(shared_aclus)}\nshared_aclus: {shared_aclus}')
# shared_aclu_neuron_type = long_session.neurons.neuron_type[np.isin(long_session.neurons.neuron_ids, shared_aclus)]
# assert len(shared_aclu_neuron_type) == len(shared_aclus)
# # Find only the aclus that are pyramidal:
# is_shared_aclu_pyramidal = (shared_aclu_neuron_type == NeuronType.PYRAMIDAL)
# pyramidal_only_shared_aclus = shared_aclus[is_shared_aclu_pyramidal]
# print(f'num pyramidal_only_shared_aclus: {len(pyramidal_only_shared_aclus)}\npyramidal_only_shared_aclus: {pyramidal_only_shared_aclus}')


## Drop Pyramidal but don't use only shared aclus:
all_aclus = deepcopy(long_session.neurons.neuron_ids)
neuron_type = long_session.neurons.neuron_type
assert len(neuron_type) == len(all_aclus)
# Find only the aclus that are pyramidal:
is_aclu_pyramidal = (neuron_type == NeuronType.PYRAMIDAL)
pyramidal_only_all_aclus = all_aclus[is_aclu_pyramidal]
print(f'num pyramidal_only_all_aclus: {len(pyramidal_only_all_aclus)}\npyramidal_only_all_aclus: {pyramidal_only_all_aclus}')

is_aclu_interneuron = (neuron_type == NeuronType.INTERNEURONS)
interneuron_only_all_aclus = all_aclus[is_aclu_interneuron]
print(f'num interneuron_only_all_aclus: {len(interneuron_only_all_aclus)}\ninterneurons: {interneuron_only_all_aclus}')

# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, shared_aclus, epoch_idx=5, callout_epoch_IDXs=[0,1,2,3], skip_rendering_callouts=True)
# app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_shared_aclus, epoch_idx=2, callout_epoch_IDXs=[0,4], skip_rendering_callouts=False)
app, win, plots, plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=6, callout_epoch_IDXs=[2,4,6], skip_rendering_callouts=False)

In [ ]:

# Use the Jupyter Index Thing
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import DiagnosticDistanceMetricFigure

## Render the internactive slider that allows selecting the timebin index to debug
n_timebins = np.sum(long_results_obj.all_epochs_num_epoch_time_bins)
timebinned_neuron_info = long_results_obj.timebinned_neuron_info
result = long_results_obj.new_result
active_fig_obj, update_function = DiagnosticDistanceMetricFigure.build_interactive_diagnostic_distance_metric_figure(long_results_obj, timebinned_neuron_info, result)
active_fig_obj.integer_slider(n_timebins=n_timebins, update_func=update_function)

In [ ]:
active_fig_obj.export()

In [ ]:
app, win, self.plots, self.plots_data = plot_kourosh_activity_style_figure(long_results_obj, long_session, pyramidal_only_all_aclus, epoch_idx=11, callout_epoch_IDXs=[0,1,2, 3, 4, 5], skip_rendering_callouts=False)

# 2023-04-13 - Find Good looking epochs:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
import matplotlib.patheffects as path_effects

from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

curr_results_obj = long_results_obj
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_pagination_controller = DecodedEpochSlicesPaginatedFigureController.init_from_decoder_data(curr_results_obj.active_filter_epochs, curr_results_obj.all_included_filter_epochs_decoder_result, 
	xbin=curr_results_obj.original_1D_decoder.xbin, global_pos_df=global_session.position.df, a_name='TestDecodedEpochSlicesPaginationController', active_context=active_identifying_session_ctx,  max_subplots_per_page=200) # 10
# _out_pagination_controller

### 2023-05-30 - Add the radon-transformed linear fits to each epoch to the stacked epoch plots:

In [ ]:
from neuropy.utils.result_context import IdentifyingContext
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.DecoderPredictionError import plot_decoded_epoch_slices_paginated

_out_pagination_controller, active_out_figure_paths, final_context = plot_decoded_epoch_slices_paginated(curr_active_pipeline, long_results_obj, curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='long_results_obj'), save_figure=True)


In [ ]:
_out_pagination_controller, active_out_figure_paths, final_context = plot_decoded_epoch_slices_paginated(curr_active_pipeline, short_results_obj, curr_active_pipeline.build_display_context_for_session(display_fn_name='DecodedEpochSlices', epochs='replays', decoder='short_results_obj'), save_figure=True)


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _prepare_plot_long_and_short_epochs

(pagination_controller_L, pagination_controller_S), (fig_L, fig_S), (ax_L, ax_S), (final_context_L, final_context_S), (active_out_figure_paths_L, active_out_figure_paths_S) = _prepare_plot_long_and_short_epochs(curr_active_pipeline, defer_render=False)

In [ ]:
## 2023-05-31 - Reference Output of matplotlib figure to file, along with building appropriate context.
active_session_figures_out_path = curr_active_pipeline.get_daily_programmatic_session_output_path()
final_context = curr_active_pipeline.sess.get_context().adding_context('display_fn', display_fn_name='DecodedEpochSlices').adding_context('display_kwargs', **dict(epochs='replays', decoder='long_results_obj'))
print(f'final_context: {final_context}')
fig = _out_pagination_controller.plots.fig # get the figure
active_out_figure_paths = perform_write_to_file(fig, final_context, figures_parent_out_path=active_session_figures_out_path, register_output_file_fn=curr_active_pipeline.register_output_file)

# Plot long|short firing rate index using 'long_short_fr_indicies_analyses':

In [ ]:
# fig_save_parent_path = Path(r'E:\Dropbox (Personal)\Active\Kamran Diba Lab\Results from 2023-01-20 - LongShort Firing Rate Indicies')
## Get the output path (active_session_figures_out_path) for this session (and all of its filtered_contexts as well):
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
figures_parent_out_path = create_daily_programmatic_display_function_testing_folder_if_needed()
active_session_figures_out_path = session_context_to_relative_path(figures_parent_out_path, active_identifying_session_ctx)
print(f'curr_session_parent_out_path: {active_session_figures_out_path}')
active_session_figures_out_path.mkdir(parents=True, exist_ok=True) # make folder if needed
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', curr_active_pipeline.sess.get_context(), fig_save_parent_path=active_session_figures_out_path)
# plt.close() # closes the current figure

In [ ]:
curr_active_pipeline.display('_display_short_long_firing_rate_index_comparison', long_short_fr_indicies_analysis_results['active_context'], fig_save_parent_path=None)


In [ ]:
import matplotlib
# configure backend here
# matplotlib.use('Qt5Agg')
# backend_qt5agg
matplotlib.use('Qt5Agg') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies


# Plot long|short firing rate index:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_dict, y_frs_dict = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
fig, _temp_full_fig_save_path = _plot_long_short_firing_rate_indicies(x_frs_dict, y_frs_dict, active_context, fig_save_parent_path=None, debug_print=False)
fig.show()

In [ ]:
ax = plt.gca()


In [ ]:
ax.spines[['top', 'right']].set_visible(True)

In [ ]:
# curr_active_pipeline.perform_specific_computation(computation_functions_name_includelist=['_perform_long_short_firing_rate_analyses'], fail_on_exception=True, debug_print=False) # fail_on_exception MUST be True or error handling is all messed up 
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_dict, y_frs_dict = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']
long_short_fr_indicies_analysis_results

# Other Programmatic Figures

### 2023-05-25 - Neptune Figure Uploads from `registered_output_files`


In [ ]:
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import set_environment_variables, neptune_output_figures

# curr_active_pipeline.registered_output_files
# curr_active_pipeline.register_output_file(output_path=test_path, output_metadata={})

neptune_output_figures(curr_active_pipeline)

In [ ]:
curr_active_pipeline.registered_output_files
# curr_active_pipeline.registered_output_files_list

In [ ]:
import matplotlib as mpl
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
mpl.rcParams['toolbar'] = 'None' # disable toolbars
matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!
# active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'
# programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=False) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.
# programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_1d_placefield_validations') # , filter_name=active_config_name 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear. Moderate visual improvements can still be made (titles overlap and stuff). Works with %%capture
# programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_result_plot_ratemaps_2D') #  🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.
programmatic_display_to_PDF(curr_active_pipeline, curr_display_function_name='_display_2d_placefield_occupancy') #  🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.


In [ ]:
curr_active_pipeline.reload_default_display_functions()
active_identifying_session_ctx, active_session_figures_out_path, active_out_figures_list = batch_programmatic_figures(curr_active_pipeline)
batch_extended_programmatic_figures(curr_active_pipeline=curr_active_pipeline)

In [ ]:
# plot_helper = curr_active_pipeline.plot
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
curr_active_pipeline.display_output_history_list

In [ ]:
curr_active_pipeline.display_output

In [ ]:

# '_display_2d_placefield_result_plot_ratemaps_2D'
# %matplotlib inline
%matplotlib qt
# curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', long_epoch_context) # MatplotlibRenderPlots

# curr_active_pipeline.get_daily_programmatic_session_output_path()
curr_active_pipeline.display_output

In [ ]:

# long_pf2D.plot_occupancy()
_out = short_one_step_decoder_2D.pf.plot_ratemaps_2D(included_unit_neuron_IDs=[2,4,5], bg_rendering_mode=BackgroundRenderingOptions.EMPTY, use_special_overlayed_title=False, missing_aclu_string_formatter=None, debug_print=False, brev_mode=PlotStringBrevityModeEnum.NONE)

## Single column output: subplots=(None, 9)


In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=False) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.

In [ ]:
long_shared_aclus_only_decoder.pf.plot_occupancy()

In [ ]:
%matplotlib qt
long_one_step_decoder_2D.pf.plot_occupancy()

In [ ]:
long_one_step_decoder_2D.pf.plot_ratemaps_2D()

In [ ]:
# curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_session_configuration_context=global_epoch_context, single_figure=False)

short_one_step_decoder_1D

In [ ]:
%matplotlib qt
active_identifying_session_ctx = curr_active_pipeline.sess.get_context() # 'bapun_RatN_Day4_2019-10-15_11-30-06'

long_single_cell_pfmap_processing_fn = None
short_single_cell_pfmap_processing_fn = None
sort_idx = None

out = curr_active_pipeline.display('_display_short_long_pf1D_comparison', active_identifying_session_ctx, single_figure=False, debug_print=False, fignum='Short v Long pf1D Comparison',
                                   long_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': long_single_cell_pfmap_processing_fn},
                                   short_kwargs={'sortby': sort_idx, 'single_cell_pfmap_processing_fn': short_single_cell_pfmap_processing_fn, 'curve_hatch_style': {'hatch':'///', 'edgecolor':'k'}},
                                  )

# ax = out.axes[0]

In [ ]:
sort_idx = np.argsort(product_overlap_scalars_df.prod_overlap.to_numpy())[::-1] # the `[::-1]` term reverses the array, which by defaul is returned in ascending order and we want descending
sort_idx

In [ ]:
curr_ratemap = long_one_step_decoder_1D.pf.ratemap
curr_ratemap.get_sort_indicies()
# .pf1D.ratemap

In [ ]:
# Create `master_dock_win` - centralized plot output window to collect individual figures/controls in (2022-08-18)

active_identifying_session_ctx = curr_active_pipeline.filtered_contexts['maze']
display_output = curr_active_pipeline.display('_display_context_nested_docks', active_identifying_session_ctx, enable_gui=False, debug_print=True) # returns {'master_dock_win': master_dock_win, 'app': app, 'out_items': out_items}
master_dock_win = display_output['master_dock_win']
app = display_output['app']
out_items = display_output['out_items']

# def _get_curr_figure_format_config():
# 	""" Aims to fetch the current figure_format_config and context from the figure_format_config widget:    
# 	Implicitly captures: `out_items`, `active_config_name`, `active_identifying_filtered_session_ctx` 
# 	"""
# 	## Get the figure_format_config from the figure_format_config widget:
# 	# Fetch the context from the GUI:
# 	_curr_gui_session_ctx, _curr_gui_out_display_items = out_items[active_config_name]
# 	_curr_gui_figure_format_config_widget = _curr_gui_out_display_items[active_identifying_filtered_session_ctx.adding_context('display_fn', display_fn_name='figure_format_config_widget')] # [0] is seemingly not needed to unpack the tuple
# 	if _curr_gui_figure_format_config_widget is not None:
# 		# has GUI for config
# 		figure_format_config = _curr_gui_figure_format_config_widget.figure_format_config
# 	else:
# 		# has non-GUI provider of figure_format_config
# 		figure_format_config = _curr_gui_figure_format_config_widget.figure_format_config

# 	if debug_print:
# 		print(f'recovered gui figure_format_config: {figure_format_config}')

# 	return figure_format_config

# figure_format_config = _get_curr_figure_format_config()

# 2023-05-25 - Radon Transform approach to finding line of best fit for each replay Epoch

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.DefaultComputationFunctions import compute_radon_transforms

## 2023-05-25 - Get the 1D Posteriors for each replay epoch so they can be analyzed via score_posterior(...) with a Radon Transform approch to find the line of best fit (which gives the velocity).
epochs_linear_fit_df_LONG = compute_radon_transforms(long_results_obj.original_1D_decoder, long_results_obj.all_included_filter_epochs_decoder_result)
assert long_results_obj.active_filter_epochs.n_epochs == np.shape(epochs_linear_fit_df_LONG)[0]
long_results_obj.active_filter_epochs._df = long_results_obj.active_filter_epochs.to_dataframe().join(epochs_linear_fit_df_LONG) # add the newly computed columns to the Epochs object
# epochs_linear_fit_df_LONG

epochs_linear_fit_df_SHORT = compute_radon_transforms(short_results_obj.original_1D_decoder, short_results_obj.all_included_filter_epochs_decoder_result)
assert short_results_obj.active_filter_epochs.n_epochs == np.shape(epochs_linear_fit_df_SHORT)[0]
short_results_obj.active_filter_epochs._df = short_results_obj.active_filter_epochs.to_dataframe().join(epochs_linear_fit_df_SHORT) # add the newly computed columns to the Epochs object
# epochs_linear_fit_df_SHORT

# Join the columns of LONG and SHORT dataframes with suffixes
assert np.shape(epochs_linear_fit_df_LONG)[0] == np.shape(epochs_linear_fit_df_SHORT)[0]
combined_epochs_linear_fit_df = epochs_linear_fit_df_LONG.join(epochs_linear_fit_df_SHORT, lsuffix='_LONG', rsuffix='_SHORT')
# combined_epochs_linear_fit_df

assert active_filter_epochs.n_epochs == np.shape(combined_epochs_linear_fit_df)[0]
active_filter_epochs._df = active_filter_epochs.to_dataframe().join(combined_epochs_linear_fit_df) # add the newly computed columns to the Epochs object
active_filter_epochs

# 2023-06-01 - Testing new metrics:

In [ ]:

curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
## Extract variables from results object:
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result['Flat_epoch_time_bins_mean'], expected_v_observed_result['Flat_decoder_time_bin_centers'], expected_v_observed_result['num_neurons'], expected_v_observed_result['num_timebins_in_epoch'], expected_v_observed_result['num_total_flat_timebins'], expected_v_observed_result['is_short_track_epoch'], expected_v_observed_result['is_long_track_epoch'], expected_v_observed_result['short_short_diff'], expected_v_observed_result['long_long_diff']


print(f'long_test: {np.mean(long_long_diff)}')
print(f'short_test: {np.mean(short_short_diff)}')

In [ ]:
long_session.replay

In [ ]:
# long_results_obj.original_1D_decoder
replay_result_df = deepcopy(long_results_obj.active_filter_epochs.to_dataframe())
replay_result_df

In [ ]:
short_results_obj.active_filter_epochs.to_dataframe()

In [ ]:
# Look at lap speed over time - doesn't look like there's a difference between the long and the short track speeds
global_session.position.compute_higher_order_derivatives()
running_pos_df = global_session.position.to_dataframe()
ax = running_pos_df.plot(x='t', y=['lin_pos','speed'], title='Running Speed over Time')
ax

In [ ]:
# Call the function
fig, ax = plot_lines(replay_result_df)

In [ ]:
import matplotlib
%matplotlib qt

_display_running_and_replay_speeds_over_time


In [ ]:
from attrs import define

In [ ]:
@define(slots=False)
class SelfContainedDisplayFunction:
    


In [ ]:
curr_active_pipeline.get_daily_programmatic_session_output_path()

# 2023-06-09 - Saving to .h5 instead of pickling

In [ ]:
curr_active_pipeline.get_session_context().to_dict()

# 2023-06-13 11:01: - [ ] New display function testing

In [15]:
import matplotlib as mpl
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import create_daily_programmatic_display_function_testing_folder_if_needed, session_context_to_relative_path, programmatic_display_to_PDF
# from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_pdf_metadata_from_display_context # newer version of build_pdf_export_metadata
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
from neuropy.plotting.ratemaps import BackgroundRenderingOptions
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

%matplotlib qt

In [30]:
curr_active_pipeline.reload_default_display_functions()


In [ ]:
# plot_helper = curr_active_pipeline.plot
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:

# '_display_2d_placefield_result_plot_ratemaps_2D'
# %matplotlib inline

# curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', long_epoch_context) # MatplotlibRenderPlots

# curr_active_pipeline.get_daily_programmatic_session_output_path()
curr_active_pipeline.display_output

In [ ]:

# long_pf2D.plot_occupancy()
_out = short_one_step_decoder_2D.pf.plot_ratemaps_2D(included_unit_neuron_IDs=[2,4,5], bg_rendering_mode=BackgroundRenderingOptions.EMPTY, use_special_overlayed_title=False, missing_aclu_string_formatter=None, debug_print=False, brev_mode=PlotStringBrevityModeEnum.NONE)

## Single column output: subplots=(None, 9)


In [ ]:
# %%capture
active_display_to_pdf_fn(curr_active_pipeline, curr_display_function_name='_display_1d_placefields', debug_print=False) # 🟢✅ Now seems to be working and saving to PDF!! Still using matplotlib.use('Qt5Agg') mode and plots still appear.

In [ ]:
long_shared_aclus_only_decoder.pf.plot_occupancy()

In [24]:
_out = curr_active_pipeline.display('_display_1d_placefields', active_session_configuration_context=global_epoch_context)
_out

MatplotlibRenderPlots({'name': 'MatplotlibRenderPlots', 'context': IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-08_21-16-25', 'maze', '1d_placefields')>, 'figures': [<Figure size 550x1100 with 1 Axes>], 'axes': [(<Axes: title={'center': '1D Placemaps Placemaps (62 good cells)'}, xlabel='Position'>, array([54, 53, 60, 28, 13, 32, 40,  4, 34, 43, 24, 15, 22,  5, 47, 21, 39,
       31,  6, 61, 35,  2, 36, 45, 51, 59, 30,  0, 20, 48, 33, 23, 12, 29,
       16, 50, 27, 19, 44, 10,  7, 56, 37, 11,  3, 18, 55, 46, 42, 57, 58,
        8, 41, 25, 26, 38,  1, 49, 17, 14, 52,  9], dtype=int64), array([[0.51764706, 0.57794118, 0.63823529, 0.69852941, 0.75882353,
        0.81911765, 0.87941176, 0.93970588, 1.        , 0.69803922,
        0.73578431, 0.77352941, 0.81127451, 0.84901961, 0.88676471,
        0.9245098 , 0.9622549 , 1.        , 0.54901961, 0.60539216,
        0.66176471, 0.71813725, 0.7745098 , 0.83088235, 0.8872549 ,
        0.94362745, 1.        , 0.38823529, 0.44845291, 0.510

In [31]:
_out = curr_active_pipeline.display('_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=global_epoch_context)
_out

MatplotlibRenderPlots({'name': 'MatplotlibRenderPlots', 'context': IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-08_21-16-25', 'maze', '_display_2d_placefield_result_plot_ratemaps_2D')>, 'figures': [<Figure size 2400x2100 with 125 Axes>], 'axes': [<mpl_toolkits.axes_grid1.axes_grid.ImageGrid object at 0x0000016CE24D0F40>], 'graphics': [{2: {'axs': [<Axes: label='2-s1, c3 - 7.5 Hz'>], 'image': <matplotlib.image.AxesImage object at 0x0000016CE67C58E0>, 'title_obj': <matplotlib.offsetbox.AnchoredText object at 0x0000016CE67C5C40>}, 3: {'axs': [<Axes: label='3-s1, c4 - 4.16 Hz'>], 'image': <matplotlib.image.AxesImage object at 0x0000016CE67D2430>, 'title_obj': <matplotlib.offsetbox.AnchoredText object at 0x0000016CE67D2790>}, 7: {'axs': [<Axes: label='7-s1, c10 - 7.86 Hz'>], 'image': <matplotlib.image.AxesImage object at 0x0000016CE67D2F40>, 'title_obj': <matplotlib.offsetbox.AnchoredText object at 0x0000016CE67DD2E0>}, 8: {'axs': [<Axes: label='8-s1, c11 - 2.66 Hz'>], 'image': <mat

In [38]:
_out = curr_active_pipeline.display('_display_2d_placefield_occupancy', active_session_configuration_context=global_epoch_context)
_out

MatplotlibRenderPlots({'name': 'MatplotlibRenderPlots', 'context': None, 'figures': [<Figure size 640x480 with 2 Axes>], 'axes': <Axes: title={'center': '(speedThresh_10.00, gridBin_2.00_2.00, smooth_2.00_2.00, frateThresh_1.00)grid_bin_bounds_((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493))'}>, 'graphics': []})

In [25]:
_out.is_matplotlib()
_out.context

IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-08_21-16-25', 'maze', '1d_placefields')>

In [44]:
active_config = curr_active_pipeline.active_configs['maze1']
plotting_config = active_config.plotting_config
computation_config = active_config.computation_config
pf_params: PlacefieldComputationParameters = computation_config.pf_params
spike_analysis: DynamicContainer = computation_config.spike_analysis
session_config = active_config.active_session_config

In [47]:
# computation_config.to_dict()

pf_params_context = IdentifyingContext.init_from_dict(pf_params.to_dict())
pf_params_context

IdentifyingContext<(10.0, (2, 2), ((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493)), (2.0, 2.0), 1.0, 0.03333, 36 epochs
array([[ 41.335214,  47.307351],
       [ 80.640746,  90.684894],
       [110.337945, 116.11017 ],
       [151.478557, 157.385648],
       [171.131812, 179.074341],
       [204.665644, 208.971027],
       [217.378244, 223.418852],
       [229.525522, 239.100362],
       [241.335915, 247.108826],
       [263.324001, 269.263814],
       [277.171262, 285.315136],
       [290.88727 , 297.259159],
       [298.794852, 307.702442],
       [313.209451, 319.148464],
       [322.617948, 329.525571],
       [342.405356, 348.110716],
       [352.415292, 358.855782],
       [371.001009, 377.272892],
       [387.349453, 393.890413],
       [399.363215, 404.534726],
       [409.03814 , 415.644351],
       [430.661146, 439.235789],
       [440.704139, 447.909801],
       [459.456427, 466.128276],
       [496.126185, 501.564096],
       [529.993019, 

In [48]:
spike_analysis_context = IdentifyingContext.init_from_dict(spike_analysis.to_dict())
spike_analysis_context

IdentifyingContext<(20000, DynamicContainer({'s': 2, 'gamma': 0.2}), False, False)>

In [49]:
# not quite right:
computation_config_context = IdentifyingContext(pf_params=pf_params_context, spike_analysis=spike_analysis_context)
computation_config_context

IdentifyingContext<(IdentifyingContext<(10.0, (2, 2), ((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493)), (2.0, 2.0), 1.0, 0.03333, 36 epochs
array([[ 41.335214,  47.307351],
       [ 80.640746,  90.684894],
       [110.337945, 116.11017 ],
       [151.478557, 157.385648],
       [171.131812, 179.074341],
       [204.665644, 208.971027],
       [217.378244, 223.418852],
       [229.525522, 239.100362],
       [241.335915, 247.108826],
       [263.324001, 269.263814],
       [277.171262, 285.315136],
       [290.88727 , 297.259159],
       [298.794852, 307.702442],
       [313.209451, 319.148464],
       [322.617948, 329.525571],
       [342.405356, 348.110716],
       [352.415292, 358.855782],
       [371.001009, 377.272892],
       [387.349453, 393.890413],
       [399.363215, 404.534726],
       [409.03814 , 415.644351],
       [430.661146, 439.235789],
       [440.704139, 447.909801],
       [459.456427, 466.128276],
       [496.126185, 501.564096],


In [37]:
plotting_config.get_figure_save_path()

WindowsPath('W:/Data/Output/2023-06-13/2006-6-08_21-16-25/maze1/speedThresh_10.00-gridBin_2.00-smooth_2.00-frateThresh_1.00-grid_bin_bounds_((15.182688599114876, 256.24088442352786), (88.93029378232772, 160.71688030454493))-time_bin_size_0.03-computation_epochs_Epoch[75](41.3-1237.5)')

In [ ]:
from enum import Enum


class FigureOutputLocation(Enum):
	"""Docstring for FigureOutputLocation."""
	DAILY_PROGRAMMATIC_OUTPUT_FOLDER = "daily_programmatic_output_folder" # the common folder for today's date
	SESSION_OUTPUT_FOLDER = "session_output_folder" # the session-specific output folder. f"{session_path}/output/figures"
	CUSTOM = "custom" # other folder. Must be specified.
	

	